In [1]:
print("test")

test


In [2]:
import binascii
import time
import os
import shutil
import numpy
import fobos
from fobos_openadc_dev import *
#WORKSPACE = "/home/xilinx/fobosworkspace"
WORKSPACE = "/mnt/nfs/fobosworkspace"
PROJECT_NAME ="aes_test"
DIN_FILE_NAME = "dinFile.txt"
CIPHER_FILE = "ciphertext.txt"
TRACE_FILE = "powerTrace.npy"
DUT_BIT_FILE = "FOBOS_DUT.bit"
TRACE_NUM = 10
OUT_LEN = 16

#program DUT


#Scope ########################################################################################################
scopConfig = {
         'OSCILLOSCOPE'       : 'AGILENT', #AGILENT|OPENADC
         'OSCILLOSCOPE_IP'    : '192.168.10.10',
         'OSCILLOSCOPE_PORT'  : '5025',
         'AUTOSCALE'          : 'NO',   # YES|NO    
         'IMPEDANCE'          : 'ONEMEG', #FIFTY|ONEMEG
         # VOLTAGE AND TIME RANGE OPTIONS        
         'CHANNEL1_RANGE'     : '0.100V',
         'CHANNEL2_RANGE'     : '6V',
         'CHANNEL3_RANGE'     : 'OFF', # ON|OFF|voltage range
         'CHANNEL4_RANGE'     : 'OFF', # ON|OFF|voltage range
         'TIME_RANGE'         :  '0.000040',
         'TIMEBASE_REF'       : 'LEFT',    
         # TRIGGER OPTIONS
         'TRIGGER_SOURCE'     : 'CHANNEL2',
         'TRIGGER_MODE'       : 'EDGE',   
         'TRIGGER_SWEEP'      : 'NORM',
         'TRIGGER_LEVEL'      : '1',
         'TRIGGER_SLOPE'      : 'POSITIVE',
         # ACQUIRE OPTIONS
         'ACQUIRE_TYPE'       : 'NORM', # NORM|PEAK|HRES|AVER
         'ACQUIRE_MODE'       : 'RTIM'   # RTIM | ETIM| SEG
}
scope = fobos.Scope()
scope.setConfig(scopConfig)
print(scope.getConfig())
scope.openConnection()
scope.applyConfig()

{'OSCILLOSCOPE': 'AGILENT', 'OSCILLOSCOPE_IP': '192.168.10.10', 'OSCILLOSCOPE_PORT': '5025', 'RESOURCE': '', 'AUTOSCALE': 'NO', 'IMPEDANCE': 'ONEMEG', 'CHANNEL1_RANGE': '0.100V', 'CHANNEL2_RANGE': '6V', 'CHANNEL3_RANGE': 'OFF', 'CHANNEL_4RANGE': '', 'CHANNEL1_DISPLAY': '', 'CHANNEL2_DISPLAY': '', 'CHANNEL3_DISPLAY': '', 'CHANNEL4_DISPLAY': '', 'TIME_RANGE': '0.000040', 'TIMEBASE_REF': 'LEFT', 'TRIGGER_SOURCE': 'CHANNEL2', 'TRIGGER_MODE': 'EDGE', 'TRIGGER_SWEEP': 'NORM', 'TRIGGER_LEVEL': '1', 'TRIGGER_SLOPE': 'POSITIVE', 'SAMPLE_INPUT': '', 'ACQUIRE_TYPE': 'NORM', 'ACQUIRE_MODE': 'RTIM', 'ACQUIRE_COMPLETE': '', 'WAVE_DATA_SIZE': '', 'NUM_PWR_TRACE': '', 'SCREEN_CAP': '', 'SCREEN_FORMAT': '', 'SCREEN_NAME': '', 'OUTPUT_DIR': '', 'CHANNEL4_RANGE': 'OFF'}


In [3]:
#dutcomm register offsets
dutcomm_START       = 0x00
dutcomm_STATUS      = 0x04
dutcomm_EXP_OUT_LEN = 0x0c
##########################
#dutctrl register offsets
dutctrl_TRGLEN     = 0x00
dutctrl_TRGWIAT    = 0x04
dutctrl_TRGMODE    = 0x08
#trigger settings
TRG_NORM            = 0X00
TRG_FULL            = 0x01
TRG_NORM_CLK        = 0x02
TRG_FULL_CLK        = 0x03
##########################
#config parameters
OUT_LEN    = 4  # in 32 bit words
TV_LEN              = 12 # in 32 bit words

##########################
import pynq.lib.dma
from pynq import Xlnk
import numpy as np
from pynq import Overlay
import sys
import time
##########################
ol = Overlay("ctrl_top_wrapper.bit")
xlnk = Xlnk()
dma = ol.axi_dma_0
#help(ol)
###set DUT clock frequency.
wiz = ol.clk_wiz
wiz.write(0x200, 0x00000102)
wiz.write(0x208, 0x00000064)
wiz.write(0x25c, 0x00000003)

dutcomm = ol.dutcomm_0
#dutcomm.read(dutcomm_STATUS)
dut_ctrl = ol.dut_controller_0
#clk_divider = ol.clock_divider_0
#configure trigger settings
#configure dutcomm expected_output_lenght
dutcomm = ol.dutcomm_0
dutcomm.write(dutcomm_EXP_OUT_LEN,4)
#set trigger to full
dut_ctrl.write(dutctrl_TRGMODE, TRG_FULL)
#clk_divider.write(0, 0xFFFFFFFF)

In [4]:
#configure OpenADC
from pynq import Xlnk
dmaRecv = ol.axi_dma_1
#dmaRecv = overlay.axi_dma_from_openadc_to_ps
openadcInterface = ol.openadc_interface_v1_0_0
adcClk = ol.clk_wiz_adc
fobosAcq = FobosAcquisition(openadcInterface, dmaRecv, adcClk)
fobosAcq.setAdcClockFreq(50)
adcClk.write(0x25c, 0x00000003)
openadcInterface.writeGain(20)

#####
xlnk = Xlnk()
outputBuffer = xlnk.cma_array(shape=(3000,), dtype=np.uint64)

clkout a01
1 10 0
clkout 1000
divInt
20
divFrac
0.0
divInt
20
divFrac
0


In [5]:
##########################
###configure worksapce
pm = fobos.ProjectManager()
pm.setWorkSpaceDir(WORKSPACE)
pm.setProjName(PROJECT_NAME)
projDir = pm.getProjDir()
##########################
dut = fobos.Nexys3DUT()
bitFile = os.path.join(projDir, DUT_BIT_FILE)
dut.setBitFile(bitFile)
dut.program()
tvFileName = os.path.join(projDir, DIN_FILE_NAME)
tvFile = open(tvFileName, "r")
captureDir = pm.getCaptureDir()
cipherFileName = os.path.join(captureDir, CIPHER_FILE)
cipherFile = open(cipherFileName, "w")
traceFileName = os.path.join(captureDir, TRACE_FILE)
traceFile = open(traceFileName, "a+b")
#print("Copying test-vector file to attempt directory ...")
#shutil.copy(tvFileName, captureDir)
#########################
TRACE_NUM = 1000
input_buffer = xlnk.cma_array(shape=(TV_LEN,), dtype=np.uint32)
output_buffer = xlnk.cma_array(shape=(OUT_LEN,), dtype=np.uint32)
##scope.arm()
time.sleep(1)
print("Loading test-vector file ...")
lines = tvFile.readlines()
print("Done loading.")
tic = time.time()
for traceNum in range(TRACE_NUM):
    #get one line of dinFile and convert every 8 characters into 32bit int
    #interpret data as hex
    line = lines[traceNum].strip()
    tv = [int(line[i:i+8],16) for i in range(0, len(line), 8)]
    #time.sleep(0.05) #this delay seem to be needed
    for i in range(0, TV_LEN):
       input_buffer[i] = tv[i]
    
    ###arm to start getting trace
    fobosAcq.arm(outputBuffer, 500)
    dma.recvchannel.transfer(output_buffer) #configure dma to receive
    dma.sendchannel.transfer(input_buffer)  #configure dma to send 
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    
    ###wait for trace to be collected
    fobosAcq.waitForTrace()
    #change data typy before saving
    trace = outputBuffer.view('uint16')
    trace= trace[:2000]
    
    #fobosAcq.getTrace(outputBuffer, blockSize=250)
    #get trigger once to use it to align
    ##if traceNum == 0:
      ##  trigTrace = scope.readChannel('CHAN2')
        #print(trigTrace)
        ###start, end = scope.getCutPoints(trigTrace)
        ###print("Cutting trace parameters: start=%d, end = %d" % (start, end))
    ##trace = scope.readChannel('CHAN1')[start:end]
    #print(trace.shape)
    #trace = trace[start:end]
    ##if traceNum != TRACE_NUM - 1: #after you finish do not arm scope. It will hang
    ##    scope.arm()
    #covert 32 bit int array into string
    result = ''.join(['{:08x}'.format(output_buffer[i]) for i in range(0, 4)])
    cipherFile.write(result + '\n')
    numpy.save(traceFile, trace)
    #if traceNum % 1000 == 0:
     #   sys.stdout.write("Progress: %d\r" % (traceNum/TRACE_NUM *100 ))
      #  sys.stdout.flush()
    #time.sleep(1)

input_buffer.freebuffer()
output_buffer.freebuffer()
cipherFile.close()
tvFile.close()
traceFile.close()
    
toc = time.time()
print("Time elapsed = {} seconds.".format(toc - tic))

Programming device using the following commands:
sudo -u xilinx djtgcfg init -d Nexys3
sudo -u xilinx djtgcfg prog -d Nexys3 -i 0 -f /mnt/nfs/fobosworkspace/aes_test/FOBOS_DUT.bit
b'Programming device. Do not touch your board. This may take a few minutes...\nProgramming succeeded.\n'
Successfully created new capture directory at /mnt/nfs/fobosworkspace/aes_test/capture/attempt-2
Loading test-vector file ...
Done loading.
Time elapsed = 9.193424701690674 seconds.


In [ ]:
import matplotlib.pyplot as plt
f = open(traceFileName, 'rb')
print(traceFileName)
dataToPlot = numpy.load(f)
plt.plot(dataToPlot)
print(dataToPlot.shape)
plt.show()